In [17]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)

import numpy as np
import pandas as pd
import warnings 
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [5]:
data = pd.read_csv('train.csv')
data.drop('Id', axis=1 , inplace=True)
data

,Comment,Topic
0,A few things. You might have negative- frequen...,Biology
1,Is it so hard to believe that there exist part...,Physics
2,There are bees,Biology
3,I'm a medication technician. And that's alot o...,Biology
4,Cesium is such a pretty metal.,Chemistry
...,...,...
8690,I make similar observations over the last week...,Biology
8691,You would know.,Biology
8692,Also use the correct number of sig figs,Chemistry
8693,"What about the ethical delimmas, groundbreaki...",Biology


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
CommentsToBeTokenized = data['Comment'] 
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()

In [8]:
vectorizer = TfidfVectorizer(max_features= 6000)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(tfidf_result, data['Topic'], test_size= 0.2)
for k in range(1,5):
    model = KNC(n_neighbors= k )
    model.fit(x_train, y_train)
    y_predicted = model.predict(x_test)
    print("\nTesting Results:\n")
    print(classification_report(y_test, y_predicted))

C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)



Testing Results:

              precision    recall  f1-score   support

     Biology       0.81      0.36      0.50       707
   Chemistry       0.39      0.88      0.54       576
     Physics       0.62      0.17      0.27       456

    accuracy                           0.48      1739
   macro avg       0.61      0.47      0.44      1739
weighted avg       0.62      0.48      0.45      1739



C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)



Testing Results:

              precision    recall  f1-score   support

     Biology       0.44      0.96      0.60       707
   Chemistry       0.49      0.13      0.21       576
     Physics       0.95      0.08      0.15       456

    accuracy                           0.45      1739
   macro avg       0.63      0.39      0.32      1739
weighted avg       0.59      0.45      0.35      1739



C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)



Testing Results:

              precision    recall  f1-score   support

     Biology       0.71      0.38      0.49       707
   Chemistry       0.39      0.89      0.55       576
     Physics       0.75      0.09      0.15       456

    accuracy                           0.47      1739
   macro avg       0.62      0.45      0.40      1739
weighted avg       0.62      0.47      0.42      1739


Testing Results:

              precision    recall  f1-score   support

     Biology       0.60      0.26      0.36       707
   Chemistry       0.36      0.62      0.45       576
     Physics       0.32      0.31      0.32       456

    accuracy                           0.39      1739
   macro avg       0.43      0.40      0.38      1739
weighted avg       0.45      0.39      0.38      1739



C:\Users\serap\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [29]:
class KNN_NLC_Classifer():
    def __init__(self, k=1, distance_type = 'path'):
        self.k = k
        self.distance_type = distance_type

    # This function is used for training
    def fit(self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train

    # This function runs the K(1) nearest neighbour algorithm and
    # returns the label with closest match. 
    def predict(self, x_test):
        self.x_test = x_test
        y_predict = []

        for i in range(len(x_test)):
            max_sim = 0
            max_index = 0
            for j in range(self.x_train.shape[0]):
                temp = self.document_similarity(x_test[i], self.x_train[j])
                if temp > max_sim:
                    max_sim = temp
                    max_index = j
            y_predict.append(self.y_train[max_index])
        return y_predict
    
    def convert_tag(self, tag):
        """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
        tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
        try:
            return tag_dict[tag[0]]
        except KeyError:
            return None


    def doc_to_synsets(self, doc):
        """
            Returns a list of synsets in document.
            Tokenizes and tags the words in the document doc.
            Then finds the first synset for each word/tag combination.
        If a synset is not found for that combination it is skipped.

        Args:
            doc: string to be converted

        Returns:
            list of synsets
        """
        tokens = word_tokenize(doc+' ')
        
        l = []
        tags = nltk.pos_tag([tokens[0] + ' ']) if len(tokens) == 1 else nltk.pos_tag(tokens)
        
        for token, tag in zip(tokens, tags):
            syntag = self.convert_tag(tag[1])
            syns = wn.synsets(token, syntag)
            if (len(syns) > 0):
                l.append(syns[0])
        return l  
    def similarity_score(self, s1, s2, distance_type = 'path'):
        """
          Calculate the normalized similarity score of s1 onto s2
          For each synset in s1, finds the synset in s2 with the largest similarity value.
          Sum of all of the largest similarity values and normalize this value by dividing it by the
          number of largest similarity values found.

          Args:
              s1, s2: list of synsets from doc_to_synsets

          Returns:
              normalized similarity score of s1 onto s2
          """
        s1_largest_scores = []

        for i, s1_synset in enumerate(s1, 0):
            max_score = 0
            for s2_synset in s2:
                if distance_type == 'path':
                    score = s1_synset.path_similarity(s2_synset, simulate_root = False)
                else:
                    score = s1_synset.wup_similarity(s2_synset)                  
                    if score != None:
                        if score > max_score:
                            max_score = score
              
                    if max_score != 0:
                        s1_largest_scores.append(max_score)
          
        mean_score = np.mean(s1_largest_scores)
                 
        return mean_score  
    def document_similarity(self,doc1, doc2):
        synsets1 = self.doc_to_synsets(doc1)
        synsets2 = self.doc_to_synsets(doc2)
          
        return (self.similarity_score(synsets1, synsets2) + self.similarity_score(synsets2, synsets1)) / 2


In [24]:
import re
s = stopwords.words('english')
ps = nltk.wordnet.WordNetLemmatizer()
for i in range(data.shape[0]):
    review = re.sub('[^a-zA-Z]', ' ', data.loc[i,'Comment'])
    review = review.lower()
    review = review.split()
    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    data.loc[i, 'Comment'] = review
X_train = data['Comment']
y_train = data['Topic']
print("Below is the sample of training text after removing the stop words")
print(data['Comment'][:10])

Below is the sample of training text after removing the stop words
0    thing might negative frequency dependent selec...
1    hard believe exist particular detect anything ...
2                                                  bee
3    medication technician alot drug liver probably...
4                                  cesium pretty metal
5                               meant question unclear
6                                 shove as see happens
7    mean butter besides sugar baking soda peanut yeah
8                         http joinchat gellhlumcxhngi
9    well thing really induce immune response intro...
Name: Comment, dtype: object


In [ ]:
classifier = KNN_NLC_Classifer(k=1, distance_type='path')
classifier.fit(X_train, y_train)

final_test_list = ['will it rain', 'Is it hot outside?' , 'What is the expected high for today?' , 
                   'Will it be foggy tomorrow?', 'Should I prepare for sleet?',
                     'Will there be a storm today?', 'do we need to take umbrella today',
                    'will it be wet tomorrow', 'is it humid tomorrow', 'what is the precipitation today',
                    'is it freezing outside', 'is it cool outside', "are there strong winds outside",]
                 
test_corpus = []
for i in range(len(final_test_list)):
    review = re.sub('[^a-zA-Z]', ' ', final_test_list[i])
    review = review.lower()
    review = review.split()

    review = [ps.lemmatize(word) for word in review if not word in s]
    review = ' '.join(review)
    test_corpus.append(review)

y_pred_final = classifier.predict(test_corpus)

output_df = pd.DataFrame(data = {'text': final_test_list, 'code': y_pred_final})
output_df['answer'] = np.where(output_df['code']==1, 'Temperature','Conditions')
print(output_df)

C:\Users\serap\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\serap\anaconda3\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
